
# Machine Translation
---
make application to translate between English and Frensh 


Steps :-
<ol>
  <li>Import Dependanices</li>
  <li>import dataset for training</li>
  <li>cleaning data</li>
  <li>tokenization and build vocabalury</li>
  <li>Pad Sequence and Vectorization</li>
  <li>Design LSTM NN (Encoder & Decoder)</li>
  <li>train model with (50,100,200) epochs</li>
  <li>build reference and Predictions</li>
</ol>

### Import Dependanices

In [ ]:
# basic libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# cleaning data
import re
import os
import nltk
nltk.download("stopwords")
nltk.download('punkt')

# save vocabulary in files
import pickle

# tokenization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model
from tensorflow.keras.layers import LSTM,Embedding,Input,Dense,SpatialDropout1D,Activation
from tensorflow.keras.models import Model,Sequential

# training model dependanices
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

### import dataset for training

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/eng_-french.csv")
df.columns=["english","frensh"]
df.head()

In [ ]:
df.info()

take first 20000 rows from data to save memory in our future we can depand on batches and make all data

In [ ]:
data=df[:]
data.info()

### cleaning data

In [ ]:
# clean english column
def clean_english(text):
  text=text.lower() # lower case

  # remove any characters not a-z and ?!,'
  text=re.sub(u"[^a-z!?',]"," ",text)

  # word tokenization
  text=nltk.word_tokenize(text)

  # join text
  text=" ".join([i.strip() for i in text])

  return text
clean_english(data.iloc[0,0])

In [ ]:
data.iloc[1,0],clean_english(data.iloc[1,0])

In [ ]:
# clean frensh language
def clean_frensh(text):
  text=text.lower() # lower case

  # remove any characters not a-z and ?!,'
  # characters a-z and (éâàçêêëôîû) chars of frensh lang which contain accent
  text=re.sub(u"[^a-zéâàçêêëôîû!?',]"," ",text)

  return text
clean_frensh(data.iloc[0,1])

In [ ]:
data.iloc[4,1],clean_frensh(data.iloc[4,1])

In [ ]:
data.iloc[6,1],clean_frensh(data.iloc[6,1])

In [ ]:
# i show this two functions are ready to apply in dataframe
data["english"]=data["english"].apply(lambda txt:clean_english(txt))
data["frensh"]=data["frensh"].apply(lambda txt:clean_frensh(txt))

In [ ]:
# add <start> <end> token to decoder sentence (Frensh)
data["frensh"]=data["frensh"].apply(lambda txt:f"<start> {txt} <end>")

In [ ]:
data.sample(10)

### tokenization and build vocabalury

In [ ]:
# english tokenizer
english_tokenize=Tokenizer(filters='#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n')
english_tokenize.fit_on_texts(data["english"])

In [ ]:
num_encoder_tokens=len(english_tokenize.word_index)
num_encoder_tokens

In [ ]:
encoder=english_tokenize.texts_to_sequences(data["english"])
encoder[:5]

In [ ]:
max_encoder_sequence_len=np.max([len(enc) for enc in encoder])
max_encoder_sequence_len

In [ ]:
# frensh tokenizer
french_tokenize=Tokenizer(filters="#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n")
french_tokenize.fit_on_texts(data["frensh"])

In [ ]:
num_decoder_tokens=len(french_tokenize.word_index)
num_decoder_tokens

In [ ]:
decoder=french_tokenize.texts_to_sequences(data["frensh"])
decoder[:5]

In [ ]:
max_decoder_sequence_len=np.max([len(dec) for dec in decoder])
max_decoder_sequence_len

In [ ]:
idx_2_txt_decoder={k:i for i,k in french_tokenize.word_index.items()}
idx_2_txt_decoder[1]

In [ ]:
idx_2_txt_encoder={k:i for i,k in english_tokenize.word_index.items()}
idx_2_txt_encoder[2]

In [ ]:
idx_2_txt_decoder[0]="<pad>"
idx_2_txt_encoder[0]="<pad>"

In [ ]:
# save idx_2_txt_encoder and idx_2_txt_decoder , english_tokenize.word_index , frensh_tokenize.word_index
pickle.dump(english_tokenize.word_index,open("/content/drive/MyDrive/word_2_idx_input.txt","wb"))
pickle.dump(french_tokenize.word_index,open("/content/drive/MyDrive/word_2_idx_target.txt","wb"))
pickle.dump(idx_2_txt_encoder,open("/content/drive/MyDrive/idx_2_word_input.txt","wb"))
pickle.dump(idx_2_txt_decoder,open("/content/drive/MyDrive/idx_2_word_target.txt","wb"))

### pad sequences 

In [ ]:
encoder_seq=pad_sequences(encoder,maxlen=max_encoder_sequence_len,padding="post")
encoder_seq.shape

In [ ]:
decoder_inp=pad_sequences([arr[:-1] for arr in decoder],maxlen=max_decoder_sequence_len,padding="post")
decoder_inp.shape

In [ ]:
decoder_output=pad_sequences([arr[1:] for arr in decoder],maxlen=max_decoder_sequence_len,padding="post")
decoder_output.shape

In [ ]:
print([idx_2_txt_decoder[i] for i in decoder_output[0]])

In [ ]:
print([idx_2_txt_encoder[i] for i in encoder_seq[0]])

In [ ]:
# decoder_output_categorical=to_categorical(decoder_output,num_classes=num_decoder_tokens+1)

In [ ]:
# decoder_output_categorical.shape

### Design LSTM NN (Encoder & Decoder)

In [ ]:
# encoder model
encoder_input=Input(shape=(None,),name="encoder_input_layer")
encoder_embedding=Embedding(num_encoder_tokens,300,input_length=max_encoder_sequence_len,name="encoder_embedding_layer")(encoder_input)
encoder_lstm=LSTM(256,activation="tanh",return_sequences=True,return_state=True,name="encoder_lstm_1_layer")(encoder_embedding)
encoder_lstm2=LSTM(256,activation="tanh",return_state=True,name="encoder_lstm_2_layer")(encoder_lstm)
_,state_h,state_c=encoder_lstm2
encoder_states=[state_h,state_c]

In [ ]:
# decoder model
decoder_input=Input(shape=(None,),name="decoder_input_layer")
decoder_embedding=Embedding(num_decoder_tokens,300,input_length=max_decoder_sequence_len,name="decoder_embedding_layer")(decoder_input)
decoder_lstm=LSTM(256,activation="tanh",return_state=True,return_sequences=True,name="decoder_lstm_layer")
decoder_outputs,_,_=decoder_lstm(decoder_embedding,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens+1,activation="softmax",name="deocer_final_layer")
outputs=decoder_dense(decoder_outputs)

In [ ]:
model=Model([encoder_input,decoder_input],outputs)
model.summary()

### train model with (50,100,200) epochs

In [ ]:
encoder_seq.shape,decoder_inp.shape,decoder_output.shape

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history=model.fit(
    [encoder_seq,decoder_inp],
    decoder_output,
    epochs=25,
    batch_size=256,
    # callbacks=[callback]
)

In [ ]:
model.save("/content/drive/MyDrive/Model_english_to_frensh.h5")

### build reference and Predictions

In [ ]:
def make_references():
  encoder_reference_model=Model(encoder_input,encoder_states)

  decoder_state_h=Input(shape=(256,))
  decoder_state_c=Input(shape=(256,))
  decoder_input_states=[decoder_state_h,decoder_state_c]

  decoder_outputs,state_h,state_c=decoder_lstm(decoder_embedding,initial_state=decoder_input_states)

  decoder_state=[state_h,state_c]
  decoder_outputs=decoder_dense(decoder_outputs)
  decoder_reference_model=Model([decoder_input]+decoder_input_states,[decoder_outputs]+decoder_state)

  return encoder_reference_model,decoder_reference_model

In [ ]:
def prepare_text(text):
  text=clean_english(text)

  res=[english_tokenize.word_index[i] for i in text.split(" ")]
  pad=pad_sequences([res],maxlen=max_encoder_sequence_len,padding="post")
  return pad
prepare_text("How are you")

In [ ]:
enc_model,dec_model=make_references()

states_value=enc_model(prepare_text(input("Enter text :- ")))

empty_target_seq=np.zeros((1,1))
empty_target_seq[0,0]=french_tokenize.word_index["start"]

stop_condition=False
decoded_translaition=""

while not stop_condition:
  dec_output,h,c=dec_model.predict([empty_target_seq]+states_value)
  sampled_word_index=np.argmax(dec_output[0,-1,:])
  sampled_word=None

  for word,index in french_tokenize.word_index.items():
    if sampled_word_index == index:
      decoded_translaition+=' {}'.format(word)
      sampled_word=word

    if sampled_word == "end" or len(decoded_translaition.split(" ")) >= max_decoder_sequence_len:
      stop_condition=True

  empty_target_seq=np.zeros((1,1))
  empty_target_seq[0,0]=sampled_word_index
  states_value=[h,c]
print(decoded_translaition)